<a href="https://colab.research.google.com/github/minju0307/kaggle_LLM_science_exam/blob/main/zero_shot_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q kaggle transformers datasets accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pimang62","key":"2f7c8dbb851487939be80745fddaa54c"}'}

In [ ]:
# 확인하기
!ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 이 일어나지 않도록
!chmod 600 ~/.kaggle/kaggle.json

# 본인이 참가한 모든 대회 보기
!kaggle competitions list

ref                                                                              deadline             category            reward  teamCount  userHasEntered  
-------------------------------------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
https://www.kaggle.com/competitions/open-problems-single-cell-perturbations      2023-11-30 23:59:00  Featured          $100,000        118           False  
https://www.kaggle.com/competitions/stanford-ribonanza-rna-folding               2023-12-07 23:59:00  Research          $100,000        104           False  
https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries         2023-10-11 23:59:00  Featured           $60,000       1651           False  
https://www.kaggle.com/competitions/bengaliai-speech                             2023-10-17 23:59:00  Research           $53,000        564           False  
https://www.kaggle.com/competitions/kaggle-llm-scien

In [ ]:
!kaggle competitions download -c kaggle-llm-science-exam

100% 72.5k/72.5k [00:00<00:00, 331kB/s]
100% 72.5k/72.5k [00:00<00:00, 330kB/s]


In [ ]:
!unzip -q kaggle-llm-science-exam.zip

# Load the data

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
train.head()

,id,prompt,A,B,C,D,E,answer
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


## Zero-shot Llama 2

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

# load base LLM model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-hf',
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf', use_auto_token=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Looking at logits for next token

In [ ]:
# Now let's apply this to multiple-choice

# First up, a way to get the prompt:
def question_from_row(row):
    prompt = f"""
    ### INSTRUCTION: ANSWER WITH A, B, C, D or E only.
    ### QUESTION: {row['prompt']}
    ### A: {row['A']}
    ### B: {row['B']}
    ### C: {row['C']}
    ### D: {row['D']}
    ### E: {row['E']}
    ### ANSWER:"""
    return prompt

# Pick a random row and create the prompt
row = train.sample(1).iloc[0]
prompt = question_from_row(row)
print("Prompt:\n", prompt)

# Now we pass it through the model and see which of the possible answer tokens is most likely
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
with torch.no_grad():
  final_logits = model(input_ids).logits[0, -1] # The last set of logits
answer_idxs = torch.tensor([tokenizer.encode(l)[-1] for l in 'ABCDE'])
answer_logits = final_logits[answer_idxs]
answer_probs = final_logits.softmax(dim=0)[answer_idxs]
print("Answer probs (all):", answer_probs)
print("Answer probs (only answer):", answer_logits.softmax(dim=0))
with torch.no_grad():
    top_k_probs, top_k_idxs = torch.topk(answer_logits, 3, dim=0)
print("Top 3", ['ABCDE'[k] for k in top_k_idxs])
print('Predicted answer:', 'ABCDE'[answer_probs.argmax()])
print('Actual answer', row['answer'])

Prompt:
 
    ### INSTRUCTION: ANSWER WITH A, B, C, D or E only.
    ### QUESTION: What is the reason that Newton's second law cannot be used to calculate the development of a physical system in quantum mechanics?
    ### A: The existence of particle spin, which is linear momentum that can be described by the cumulative effect of point-like motions in space.
    ### B: The existence of particle spin, which is angular momentum that is always equal to zero.
    ### C: The existence of particle spin, which is linear momentum that cannot be described by the cumulative effect of point-like motions in space.
    ### D: The existence of particle spin, which is angular momentum that cannot be described by the cumulative effect of point-like motions in space.
    ### E: The existence of particle spin, which is angular momentum that can be described by the cumulative effect of point-like motions in space.
    ### ANSWER:


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


Answer probs (all): tensor([0.1039, 0.1585, 0.1754, 0.1782, 0.1115])
Answer probs (only answer): tensor([0.1429, 0.2178, 0.2411, 0.2449, 0.1533])
Top 3 ['D', 'C', 'B']
Predicted answer: D
Actual answer D


In [ ]:
# See how accurate this is
from tqdm.auto import tqdm

def question_from_row(row):
    prompt = f"""
    ### INSTRUCTION: ANSWER WITH A, B, C, D or E only.
    ### QUESTION: {row['prompt']}
    ### A: {row['A']}
    ### B: {row['B']}
    ### C: {row['C']}
    ### D: {row['D']}
    ### E: {row['E']}
    ### ANSWER: """
    return prompt

first = 0
second = 0
third = 0
total = 0

for i, row in tqdm(train.iterrows(), total=len(train)):
    prompt = question_from_row(row)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    with torch.no_grad():
      final_logits = model(input_ids).logits[0, -1] # The last set of logits
    answer_idxs = answer_idxs = torch.tensor([tokenizer.encode(l)[-1] for l in 'ABCDE'])
    answer_logits = final_logits[answer_idxs]
    answer = row['answer']
    answer_probs = answer_logits.softmax(dim=0)


    with torch.no_grad():
        top_k_probs, top_k_idxs = torch.topk(answer_logits, 3, dim=0)

    if i < 5: # Some debugging
      print("Prompt:\n", prompt)
      print("Answer probs:", answer_probs)
      print("Top 3", ['ABCDE'[k] for k in top_k_idxs])
      print('Predicted answer:', 'ABCDE'[answer_probs.argmax()])
      print('Actual answer', answer)
    if 'ABCDE'[top_k_idxs[0]] == answer:
        first += 1
    if 'ABCDE'[top_k_idxs[1]] == answer:
        second += 1
    if 'ABCDE'[top_k_idxs[2]] == answer:
        third += 1
    total += 1

print("\n\nAccuracy @1:", first/total)
print("MAP@3: ", (first + second/2 + third/3) /total )

  0%|          | 0/200 [00:00<?, ?it/s]

Prompt:
 
    ### INSTRUCTION: ANSWER WITH A, B, C, D or E only.
    ### QUESTION: Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?
    ### A: MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."
    ### B: MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.
    ### C: MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.
    ### D: MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersi

## Perplexity

In [ ]:
input_ids = tokenizer("This is a very likely string", return_tensors="pt").input_ids
with torch.no_grad():
  final_logits = model(input_ids).logits
final_logits.shape # One logit per token (32000) for each token in the string

torch.Size([1, 7, 32000])

In [ ]:
for token_logits, token in zip(final_logits[0], input_ids[0][1:]): # Note the shift by 1
  print(token_logits.softmax(dim=0)[token]) # The probability for that particular token

tensor(0.0002)
tensor(0.1503)
tensor(0.4027)
tensor(0.0281)
tensor(5.5973e-05)
tensor(6.2881e-05)


In [ ]:
def calculate_perplexity(s):
  input_ids = tokenizer(s, return_tensors="pt").input_ids
  with torch.no_grad():
    final_logits = model(input_ids).logits
  perplexity = []
  for token_logits, token in zip(final_logits[0], input_ids[0][1:]):
    perplexity.append(-torch.log(token_logits.softmax(dim=0)[token])) # Cross entropy: -log(p(correct_answer))
  perplexity = torch.stack(perplexity, dim=0)
  perplexity = torch.mean(perplexity)
  return perplexity.cpu().item()

calculate_perplexity("This is a very likely string"),\
calculate_perplexity("This do string likely a")

(5.742154598236084, 10.103311538696289)

In [ ]:
row = train.sample(1).iloc[0]
perplexities = []

for possible_answer in 'ABCDE':
    prompt = f"""
    ### QUESTION: {row['prompt']}
    ### ANSWER: {row[possible_answer]}"""
    perplexity = calculate_perplexity(prompt)
    perplexities.append(perplexity)
    print(f"Perplexity for {possible_answer}: {perplexity}")

print("Correct answer:", row['answer'])
print("Perplexities:", perplexities)
print("Predicted answer:", 'ABCDE'[perplexities.index(min(perplexities))])

Perplexity for A: 2.7240407466888428
Perplexity for B: 2.6063685417175293
Perplexity for C: 2.635423183441162
Perplexity for D: 2.61726713180542
Perplexity for E: 2.41337513923645
Correct answer: C
Perplexities: [2.7240407466888428, 2.6063685417175293, 2.635423183441162, 2.61726713180542, 2.41337513923645]
Predicted answer: E


In [ ]:
# Apply this to all questions, scoring the same way we did with the previous approach

first = 0
second = 0
third = 0
total = 0

predictions = []  # 정답 리스트
for i, row in tqdm(train.iterrows(), total=len(train)):
    perplexities = []

    for possible_answer in 'ABCDE':
        prompt = f"""### QUESTION: {row['prompt']}
                     ### ANSWER: {row[possible_answer]}"""
        perplexity = calculate_perplexity(prompt)
        perplexities.append(perplexity)

    top_k_probs, top_k_idxs = torch.topk(-torch.tensor(perplexities), 3, dim=0) # - so we get the three lowest perplexities

    predictions.append(' '.join(['ABCDE'[top_k_idxs[i]] for i in range(3)]))

    if 'ABCDE'[top_k_idxs[0]] == row['answer']:
        first += 1
    if 'ABCDE'[top_k_idxs[1]] == row['answer']:
        second += 1
    if 'ABCDE'[top_k_idxs[2]] == row['answer']:
        third += 1
    total += 1

print("\n\nAccuracy @1:", first/total)
print("MAP@3: ", (first + second/2 + third/3) /total)

predictions[:10]

  0%|          | 0/200 [00:00<?, ?it/s]



Accuracy @1: 0.505
MAP@3:  0.6341666666666667


['A E D',
 'C B D',
 'B E D',
 'B C D',
 'D B A',
 'D C E',
 'C D A',
 'E A C',
 'B C A',
 'E C D',
 'C A D',
 'C D E',
 'A E D',
 'E A D',
 'D E C',
 'E D C',
 'A C E',
 'C E B',
 'A D B',
 'E B A',
 'E B A',
 'A D C',
 'C D E',
 'E D B',
 'D E B',
 'E A B',
 'B C D',
 'D B A',
 'B E C',
 'C B A',
 'B D E',
 'C A E',
 'A E B',
 'D B A',
 'A C D',
 'B A E',
 'E A D',
 'A C E',
 'E A D',
 'B D E',
 'E A B',
 'E C D',
 'C B E',
 'D C B',
 'D C A',
 'C E B',
 'B E D',
 'E D B',
 'C D B',
 'B D E',
 'A C E',
 'A E D',
 'C B E',
 'B E C',
 'B A E',
 'B E C',
 'C A D',
 'D A C',
 'D C B',
 'B A C',
 'C A E',
 'B C E',
 'B E C',
 'E A C',
 'D A C',
 'C D B',
 'E B A',
 'E C A',
 'B E C',
 'C E A',
 'E C A',
 'E D A',
 'C D E',
 'D B A',
 'D C A',
 'D B C',
 'D A C',
 'B D C',
 'C D A',
 'E C D',
 'C D A',
 'B A E',
 'B C E',
 'A B C',
 'B E A',
 'D B C',
 'D B E',
 'B D A',
 'E A B',
 'C E B',
 'C A D',
 'A E D',
 'B E C',
 'D A E',
 'E B C',
 'C E A',
 'D B E',
 'B D C',
 'D B A',
 'D C B',


## Inference

In [ ]:
test = pd.read_csv('test.csv')
test.head()

,id,prompt,A,B,C,D,E
0,0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...
1,1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...
2,2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...
3,3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...
4,4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...


In [ ]:
prediction_test = []  # 정답 리스트
for i, row in tqdm(test.iterrows(), total=len(test)):
    perplexities = []

    for possible_answer in 'ABCDE':
        prompt = f"""### QUESTION: {row['prompt']}
                     ### ANSWER: {row[possible_answer]}"""
        perplexity = calculate_perplexity(prompt)
        perplexities.append(perplexity)

    top_k_probs, top_k_idxs = torch.topk(-torch.tensor(perplexities), 3, dim=0) # - so we get the three lowest perplexities

    prediction_test.append(' '.join(['ABCDE'[top_k_idxs[i]] for i in range(3)]))

prediction_test[:10]

  0%|          | 0/200 [00:00<?, ?it/s]

['A E D',
 'C B D',
 'B E D',
 'B C D',
 'D B A',
 'D C E',
 'C D A',
 'E A C',
 'B C A',
 'E C D']

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission.head()

,id,prediction
0,0,A B C
1,1,A B C
2,2,A B C
3,3,A B C
4,4,A B C


In [ ]:
submission['prediction'] = predictions
submission.head()

,id,prediction
0,0,A E D
1,1,C B D
2,2,B E D
3,3,B C D
4,4,D B A


In [ ]:
submission.to_csv('submission_zero_shot_llama2.csv', index=False)